## 1. Importação de Bibliotecas e Configurações

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import pickle
import ast
from joblib import load
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    accuracy_score, precision_score, recall_score, 
    f1_score, roc_auc_score, roc_curve
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configurações de plotagem
plt.rcParams['figure.figsize'] = [15, 10]
sns.set_style("whitegrid")

print("Bibliotecas importadas com sucesso!")

## 2. Recuperação dos Resultados dos Modelos Treinados

In [ ]:
def load_model_results(results_folder='modeling/results'):
    """Carrega todos os resultados de avaliação salvos dos notebooks individuais"""
    all_results = {}

    if not os.path.exists(results_folder):
        print(f"Pasta {results_folder} não encontrada!")
        print("Execute os notebooks individuais dos modelos primeiro.")
        return all_results

    # Mapear arquivos para modelos
    # Comentar as linhas dos modelos que você não deseja carregar
    result_files = {
        'knn_results.json': 'KNN',
        'lvq_results.json': 'LVQ', 
        'svm_results.json': 'SVM',
        'rf_results.json': 'Random Forest',
        'dt_results.json': 'Decision Tree'
    }

    for filename, model_name in result_files.items():
        filepath = os.path.join(results_folder, filename)
        
        if os.path.exists(filepath):
            try:
                with open(filepath, 'r') as f:
                    result_data = json.load(f)
                all_results[model_name] = result_data
                print(f"{model_name}: Carregado com sucesso")
            except Exception as e:
                print(f"Erro ao carregar {filename}: {e}")
        else:
            print(f"Arquivo não encontrado: {filename}")

    return all_results

In [ ]:
final_results = load_model_results()

## 3. Preparação dos Dados para Plotagem

In [ ]:
# Criar DataFrame com métricas dos modelos para plotagem
def create_metrics_dataframe(final_results):
    """Cria DataFrame com as métricas de todos os modelos para facilitar plotagem"""
    metrics_data = []
    
    for model_name, results in final_results.items():
        train_metrics = results['train_metrics']
        test_metrics = results['test_metrics']
        
        metrics_data.append({
            'Modelo': model_name,
            'Acurácia_Teste': test_metrics['accuracy_score'],
            'F1_CV': results.get('best_cv_score', 0.0),  # Score do CV (melhor configuração)
            'F1_Treino': train_metrics['f1'],
            'F1_Teste': test_metrics['f1'],
            'Precisão_Teste': test_metrics['precision'],
            'Recall_Teste': test_metrics['recall'],
            'G-Mean_Teste': test_metrics['gmean']
        })
    
    return pd.DataFrame(metrics_data)

# Criar o DataFrame de métricas
if final_results:
    metrics_df = create_metrics_dataframe(final_results)
    print(f"Modelos disponíveis: {list(final_results.keys())}")
    print(f"\nPrimeiras linhas do DataFrame:")
    print(metrics_df.head())
else:
    print("Nenhum resultado encontrado")

## 4. Gráficos de Comparação de Métricas

In [ ]:
# 6.1 Gráficos de comparação
fig, axes = plt.subplots(1, 3, figsize=(20, 8))

# Gráfico 1: Comparação de F1-Score
ax1 = axes[0]
x_pos = np.arange(len(metrics_df))
ax1.bar(x_pos - 0.2, metrics_df['F1_CV'], 0.4, label='F1-Score CV', alpha=0.8)
ax1.bar(x_pos + 0.2, metrics_df['F1_Teste'], 0.4, label='F1-Score Teste', alpha=0.8)
ax1.set_xlabel('Modelos')
ax1.set_ylabel('F1-Score')
ax1.set_title('Comparação F1-Score: CV vs Teste')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(metrics_df['Modelo'], rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Comparação de múltiplas métricas
ax2 = axes[1]
metrics_to_plot = ['Acurácia_Teste', 'Precisão_Teste', 'Recall_Teste']
x_pos = np.arange(len(metrics_df))
width = 0.2

for i, metric in enumerate(metrics_to_plot):
    ax2.bar(x_pos + (i-1.5)*width, metrics_df[metric], width, 
            label=metric.replace('_Teste', ''), alpha=0.8)

ax2.set_xlabel('Modelos')
ax2.set_ylabel('Score')
ax2.set_title('Comparação de Múltiplas Métricas')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(metrics_df['Modelo'], rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gráfico 3: Boxplot para verificar overfitting
ax3 = axes[2]
overfitting_data = []
models_names = []
for model_name, results in final_results.items():
    f1_train = results['train_metrics']['f1']
    f1_test = results['test_metrics']['f1']
    overfitting_data.append([f1_train, f1_test])
    models_names.append(model_name)

overfitting_df = pd.DataFrame(overfitting_data, 
                             columns=['Treino', 'Teste'], 
                             index=models_names)

overfitting_df.plot(kind='bar', ax=ax3, alpha=0.8)
ax3.set_title('Overfitting: F1-Score Treino vs Teste')
ax3.set_ylabel('F1-Score')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

## 5. Matrizes de Confusão

In [ ]:
def plot_confusion_matrices(final_results):
    """Plota matrizes de confusão individuais para cada modelo - adaptado para SVHN (10 classes)"""
    
    if not final_results:
        print("❌ Nenhum resultado disponível para plotar matrizes de confusão.")
        return
    
    # Obter os labels verdadeiros da primeira predição disponível
    first_model_name = next(iter(final_results.keys()))
    y_test = final_results[first_model_name]['test_labels']
    
    print(f"📊 qPlotando matrizes de confusão para {len(final_results)} modelo(s)")
    print(f"🎯 Dataset SVHN: 10 classes (dígitos 0-9)")
    print(f"📈 Total de amostras de teste: {len(y_test):,}")
    print("=" * 60)
    
    # Plotar matrizes de confusão - uma por modelo
    for model_name, results in final_results.items():
        y_pred = results['predictions']
        
        # Matriz de confusão
        cm = confusion_matrix(y_test, y_pred)
        
        # Criar figura individual para cada matriz
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    square=True, linewidths=0.5, cbar_kws={"shrink": .8})
        
        plt.title(f'Matriz de Confusão - {model_name}', 
                 fontsize=16, fontweight='bold', pad=20)
        plt.xlabel('Classe Predita', fontsize=12, fontweight='bold')
        plt.ylabel('Classe Real', fontsize=12, fontweight='bold')
        
        # Labels das classes SVHN (dígitos 0-9)
        class_labels = [f'Dígito {i}' for i in range(10)]
        plt.xticks(np.arange(10) + 0.5, range(10), rotation=0)
        plt.yticks(np.arange(10) + 0.5, range(10), rotation=0)
        
        plt.tight_layout()
        plt.show()
        
        # Estatísticas detalhadas da matriz de confusão
        accuracy = np.trace(cm) / np.sum(cm)
        total_samples = np.sum(cm)
        
        print(f"\\n📊 {model_name} - Detalhes da Matriz de Confusão:")
        print(f"  ✅ Predições Corretas: {np.trace(cm):,}")
        print(f"  ❌ Predições Incorretas: {total_samples - np.trace(cm):,}")
        print(f"  📈 Taxa de Acerto: {accuracy:.1%}")
        print(f"  📊 Total de amostras: {total_samples:,}")
        
        # Mostrar classes mais confundidas
        np.fill_diagonal(cm, 0)  # Remover diagonal para ver apenas erros
        max_confusion_idx = np.unravel_index(np.argmax(cm), cm.shape)
        max_confusion_value = cm[max_confusion_idx]
        
        if max_confusion_value > 0:
            print(f"  🔄 Maior confusão: Dígito {max_confusion_idx[0]} → Dígito {max_confusion_idx[1]} ({max_confusion_value} casos)")
        
        print("=" * 60)

# Plotar matrizes se temos dados
if final_results:
    plot_confusion_matrices(final_results)
else:
    print("⚠️  Aguardando resultados dos modelos para plotar matrizes de confusão.")

## 6. Visualização Compacta de Todas as Matrizes

In [ ]:
def plot_all_confusion_matrices_grid(final_results):
    """Plota todas as matrizes de confusão em uma única figura (estilo grid)"""
    
    if not final_results:
        print("❌ Nenhum resultado disponível.")
        return
    
    n_models = len(final_results)
    
    # Calcular layout do grid
    if n_models <= 2:
        rows, cols = 1, n_models
        figsize = (6*n_models, 6)
    elif n_models <= 4:
        rows, cols = 2, 2
        figsize = (12, 12)
    else:
        rows = (n_models + 2) // 3
        cols = 3
        figsize = (18, 6*rows)
    
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    fig.suptitle('Matrizes de Confusão - Todos os Modelos (SVHN)', fontsize=16, fontweight='bold')
    
    # Se há apenas um modelo, axes não é uma lista
    if n_models == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes if hasattr(axes, '__iter__') else [axes]
    else:
        axes = axes.flatten()
    
    # Obter labels verdadeiros
    first_model_name = next(iter(final_results.keys()))
    y_test = final_results[first_model_name]['test_labels']
    
    model_idx = 0
    for model_name, results in final_results.items():
        y_pred = results['predictions']
        cm = confusion_matrix(y_test, y_pred)
        
        ax = axes[model_idx]
        
        # Plot da matriz
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   square=True, linewidths=0.5, 
                   cbar_kws={"shrink": .8}, ax=ax)
        
        ax.set_title(f'{model_name}', fontweight='bold', fontsize=12)
        ax.set_xlabel('Predito', fontsize=10)
        ax.set_ylabel('Real', fontsize=10)
        
        # Labels menores para economizar espaço
        ax.set_xticklabels(range(10), fontsize=8)
        ax.set_yticklabels(range(10), fontsize=8, rotation=0)
        
        model_idx += 1
    
    # Esconder axes extras se houver
    for idx in range(model_idx, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Resumo comparativo
    print("\\n📊 Resumo Comparativo - Acurácia por Classe:")
    print("=" * 70)
    
    # Criar uma tabela de acurácia por classe
    class_accuracies = {}
    
    for model_name, results in final_results.items():
        y_pred = results['predictions']
        cm = confusion_matrix(y_test, y_pred)
        
        # Calcular acurácia por classe (diagonal / soma da linha)
        class_acc = []
        for i in range(10):
            if cm[i].sum() > 0:
                acc = cm[i, i] / cm[i].sum()
                class_acc.append(acc)
            else:
                class_acc.append(0.0)
        
        class_accuracies[model_name] = class_acc
    
    # Converter para DataFrame para visualização
    acc_df = pd.DataFrame(class_accuracies, index=[f'Dígito {i}' for i in range(10)])
    print(acc_df.round(3))

# Plotar grid compacto se temos dados
if final_results:
    plot_all_confusion_matrices_grid(final_results)
else:
    print("⚠️  Aguardando resultados dos modelos para plotar grid de matrizes.")

## 7. Relatório Final de Resultados

In [ ]:
def generate_final_report(final_results, metrics_df):
    """Gera um relatório final consolidado com ranking dos modelos"""
    
    if not final_results or metrics_df.empty:
        print("❌ Não há dados suficientes para gerar o relatório.")
        return
    
    # Ranking por F1-Score no teste
    ranking_f1 = metrics_df.sort_values('F1_Teste', ascending=False)
    
    print("\n🥇 RANKING POR F1-SCORE (TESTE):")
    print("-" * 40)
    
    for idx, (_, row) in enumerate(ranking_f1.iterrows(), 1):
        medal = "🥇" if idx == 1 else "🥈" if idx == 2 else "🥉" if idx == 3 else f"{idx}º"
        print(f"{medal} {row['Modelo']}: {row['F1_Teste']:.4f}")
    
    # Ranking por Acurácia no teste
    ranking_acc = metrics_df.sort_values('Acurácia_Teste', ascending=False)
    
    print("\n🎯 RANKING POR ACURÁCIA (TESTE):")
    print("-" * 40)
    
    for idx, (_, row) in enumerate(ranking_acc.iterrows(), 1):
        medal = "🥇" if idx == 1 else "🥈" if idx == 2 else "🥉" if idx == 3 else f"{idx}º"
        print(f"{medal} {row['Modelo']}: {row['Acurácia_Teste']:.4f}")
    
    
    # Melhor modelo geral
    print("\n🏆 MODELO RECOMENDADO:")
    print("-" * 25)
    
    # Critério: melhor F1 no teste com menor overfitting
    best_model_idx = ranking_f1.index[0]
    best_model = ranking_f1.loc[best_model_idx]
    
    print(f"🥇 {best_model['Modelo']}")
    print(f"   📈 F1-Score (Teste): {best_model['F1_Teste']:.4f}")
    print(f"   🎯 Acurácia (Teste): {best_model['Acurácia_Teste']:.4f}")
    print(f"   📊 Precisão: {best_model['Precisão_Teste']:.4f}")
    print(f"   🔍 Recall: {best_model['Recall_Teste']:.4f}")
    
    # Informações adicionais do melhor modelo
    best_model_name = best_model['Modelo']
    if best_model_name in final_results:
        best_results = final_results[best_model_name]
        
        print(f"\n🔧 CONFIGURAÇÃO DO MELHOR MODELO:")
        print("-" * 35)
        if 'best_params' in best_results:
            for param, value in best_results['best_params'].items():
                print(f"   {param}: {value}")
        
        if 'evaluation_info' in best_results:
            eval_info = best_results['evaluation_info']
            print(f"\n📊 INFORMAÇÕES DE AVALIAÇÃO:")
            print("-" * 30)
            print(f"   Amostras de treino: {eval_info.get('train_samples_used', 'N/A'):,}")
            print(f"   Amostras de teste: {eval_info.get('test_samples_used', 'N/A'):,}")
    
    # Análise de desempenho por classe (se disponível)
    print(f"\n📋 ANÁLISE DE DESEMPENHO POR CLASSE:")
    print("-" * 45)
    
    if best_model_name in final_results:
        best_results = final_results[best_model_name]
        y_test = best_results['test_labels']
        y_pred = best_results['predictions']
        
        # Calcular métricas por classe
        from sklearn.metrics import classification_report
        report = classification_report(y_test, y_pred, output_dict=True)
        
        print("Desempenho por dígito (F1-Score):")
        for digit in range(10):
            if str(digit) in report:
                f1_digit = report[str(digit)]['f1-score']
                support = report[str(digit)]['support']
                status = "🟢" if f1_digit > 0.8 else "🟡" if f1_digit > 0.6 else "🔴"
                print(f"   Dígito {digit}: {f1_digit:.3f} ({support:,} amostras) {status}")
        
        # Macro e weighted averages
        print(f"\n   📊 Macro F1: {report['macro avg']['f1-score']:.4f}")
        print(f"   ⚖️  Weighted F1: {report['weighted avg']['f1-score']:.4f}")
    
    print("\n" + "=" * 60)
    print("✅ Relatório gerado com sucesso!")

# Gerar relatório final se temos dados
if final_results and not metrics_df.empty:
    generate_final_report(final_results, metrics_df)
else:
    print("⚠️  Execute os notebooks dos modelos primeiro para gerar o relatório.")